In [ ]:
import pandas as pd
from tqdm import tqdm
from datetime import timedelta
import math

In [1]:
# Load and parse the S&P 500 data (month/day/year format)
sp500 = pd.read_csv("Ctrl_Var/s&p500.csv")

# Parse dates correctly
sp500['Date'] = pd.to_datetime(sp500['Date'], format="%m/%d/%Y")

# Sort and set index
sp500 = sp500.sort_values('Date')
sp500.set_index('Date', inplace=True)

# Collect results
results = []
years = ['2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023']

for year in years:
    # Load company filing data (day/month/year format)
    current_year_df = pd.read_csv(f'Miguel Mapping/CAR_Filtered/CAR_Filtered/CAR_{year}_filtered.csv')

    # Convert 'Filing Date' to datetime (dayfirst=True for European format)
    current_year_df['Filing Date'] = pd.to_datetime(current_year_df['Filing Date'], dayfirst=True)

    for _, row in tqdm(
        current_year_df.iterrows(), 
        total=current_year_df.shape[0],
        desc=f"Processing {year}",
        unit="row", ncols=100, mininterval=0.1
    ):
        filing_date = row['Filing Date']
        cik = row['CIK']

        eventday_minus_1 = filing_date - timedelta(days=1)
        eventday_plus_1 = filing_date + timedelta(days=1) 
        eventday_plus_30 = filing_date + timedelta(days=30)
        eventday_plus_60 = filing_date + timedelta(days=60)
        eventday_plus_90 = filing_date + timedelta(days=90)



        # Shift one trading day forward to compute return (t+1 and t)
        try:
            # Ensure filing_date is in sp500 index (else get next available trading day)
            if eventday_minus_1 not in sp500.index:
                while eventday_minus_1 not in sp500.index:
                    eventday_minus_1 -= timedelta(days=1)
            
            if eventday_plus_1 not in sp500.index:
                while eventday_plus_1 not in sp500.index:
                    eventday_plus_1 -= timedelta(days=1)
                    
            if eventday_plus_30 not in sp500.index:
                while eventday_plus_30 not in sp500.index:
                    eventday_plus_30 -= timedelta(days=1)
            
            if eventday_plus_60 not in sp500.index:
                while eventday_plus_60 not in sp500.index:
                    eventday_plus_60 -= timedelta(days=1)
                    
            if eventday_plus_90 not in sp500.index:
                while eventday_plus_90 not in sp500.index:
                    eventday_plus_90 -= timedelta(days=1)
                
                event_price_minus_1 = sp500.loc[eventday_minus_1, 'Close/Last']
                event_price_plus_1 = sp500.loc[eventday_plus_1, 'Close/Last']
                event_price_plus_30 = sp500.loc[eventday_plus_30, 'Close/Last']
                event_price_plus_60 = sp500.loc[eventday_plus_60, 'Close/Last']
                event_price_plus_90 = sp500.loc[eventday_plus_90, 'Close/Last']
                
            else:   
                event_price_minus_1 = sp500.loc[eventday_minus_1, 'Close/Last']
                event_price_plus_1 = sp500.loc[eventday_plus_1, 'Close/Last']
                event_price_plus_30 = sp500.loc[eventday_plus_30, 'Close/Last']
                event_price_plus_60 = sp500.loc[eventday_plus_60, 'Close/Last']
                event_price_plus_90 = sp500.loc[eventday_plus_90, 'Close/Last']


            # Raw Return
            return_1 = (event_price_plus_1 - event_price_minus_1) / event_price_minus_1
            return_30 = (event_price_plus_30 - event_price_minus_1) / event_price_minus_1
            return_60 = (event_price_plus_60 - event_price_minus_1) / event_price_minus_1
            return_90 = (event_price_plus_90 - event_price_minus_1) / event_price_minus_1
            
            # Log Return
            log_01_day_return = math.log(event_price_plus_1 / event_price_minus_1)
            log_30_day_return = math.log(event_price_plus_30 / event_price_minus_1)
            log_60_day_return = math.log(event_price_plus_60 / event_price_minus_1)
            log_90_day_return = math.log(event_price_plus_90 / event_price_minus_1)
            
            
            
            results.append({
                "CIK": cik,
                "Filing Date": filing_date.date(),
                "S&P500 1 Day Return": return_1,
                "S&P500 30 Day Return": return_30,
                "S&P500 60 Day Return": return_60,
                "S&P500 90 Day Return": return_90,
                
                "S&P500 Log 1 Day Return": log_01_day_return,
                "S&P500 Log 30 Day Return": log_30_day_return,
                "S&P500 Log 60 Day Return": log_60_day_return,
                "S&P500 Log 90 Day Return": log_90_day_return
                
            })

        except Exception as e:
            print(f"Skipped {cik} on {filing_date.date()} — {e}")
            continue

# Convert result to DataFrame
return_df = pd.DataFrame(results)

# Save to CSV
return_df.to_csv("Ctrl_Var/sp500_returns.csv", index=False)

print("✅ Done! Returns calculated.")


Processing 2023: 100%|█████████████████████████████████████| 14417/14417 [00:03<00:00, 4205.38row/s]


✅ Done! Returns calculated.
